# Hard particle simulations with HOOMD-blue

* Introduction to HOOMD-blue.
  * The Simulation object and operations.
* Hard Spheres.
  * Initialize, compress, equilibrate, and analyze.
* Hard Spherocyliners.
* Homework problems.
  * [Homework 1 - Hard Spheres](01-Homework-Hard-Spheres.ipynb)
  * [Homework 2 - Hard Spherocylinders](01-Homework-Hard-Spheres.ipynb)

# The Simulation Object

## Overview

### Questions

* How can I configure and control a simulation?
* How do I choose which processor to use?

### Objectives

* Explain the parts of the **Simulation** object and how they relate.
* Explain how the **device** object influences how the simulation executes.
* Demonstrate the creation of these objects.

## Boilerplate code

In [ ]:
# This cell installs HOOMD in Google Colab. Delete it if you run locally
!pip install -q condacolab
import condacolab
condacolab.install_from_url('https://github.com/joaander/hoomd-che629/releases/download/2022.0.0-beta.1/hoomd-che629-2022.0-Linux-x86_64.sh')

In [ ]:
import hoomd
import math
import itertools
import numpy
import copy
import gsd.hoomd
import freud
import matplotlib
import IPython
import rowan
%matplotlib inline
matplotlib.style.use('ggplot')

IMAGE_W=250
IMAGE_SAMPLES=100
# IMAGE_W=500
# IMAGE_SAMPLES=800

## Core objects

HOOMD-blue is an object-oriented Python package. First, import the package:

In [ ]:
import hoomd

The **Simulation** object combines all the elements of a simulation together and provides an interface to run the simulation. It consists of the simulation **state** and **operations** which act on that state. The simulation **state** includes the current box, bonds, particle positions, velocities, orientations, and other particle properties. **Operations** examine or modify the state. A simulation has *one* state, and *any number* of operations.

## Selecting a device

You must specify a **device** when constructing a **Simulation**. The **device** tells the simulation where to store the **state** and what processor to use when executing operations. HOOMD-blue can execute on the CPU:

In [ ]:
cpu = hoomd.device.CPU()

Or on a GPU (if available) with `hoomd.device.GPU()`:

GPUs are highly parallel processors best suited for larger workloads.
In typical systems with more than 4,000 particles a single GPU performs an order of magnitude faster than an entire CPU (all cores).
Try your simulation on both to see what hardware can run the same number of time steps in less time.

## Creating a Simulation

Now, you can instantiate a **Simulation** with the chosen device.

In [ ]:
sim = hoomd.Simulation(device=cpu)

A newly constructed **Simulation** has no **state**:

In [ ]:
print(sim.state)

And no integrator, updaters, or writers, which are types of **operations**:

In [ ]:
print(sim.operations.integrator)

In [ ]:
print(sim.operations.updaters[:])

In [ ]:
print(sim.operations.writers[:])

The remaining sections in this tutorial will show you how to populate a **Simulation** with **operations**, initialize the **state**, and run the simulation.

# Performing Hard Particle Monte Carlo Simulations

## Overview

### Questions

* What is hard particle Monte Carlo?
* How do I set up a hard particle Monte Carlo simulation?

### Objectives

* Describe hard particle Monte Carlo simulations, **particle shape**, and **trial moves**.
* Show how to initialize the **Sphere integrator**.
* Explain the integrator parameters.
* Introduce **time steps**.

## Particle shape

A hard particle Monte Carlo (HPMC) simulation represents particles as extended objects which are not allowed to overlap. 
There are no attractive or repulsive forces in the system.
The **shape** of the particle alone controls how it interacts with other particles.
Formally, the potential energy of the system is zero when there are no overlaps and infinite when there are.
Purely hard interactions induce *effective attractions* between particles which can lead to ordered structures.
For example, hard spheres will self-assemble into an fcc structure. 
In this tutorial, you will learn how to run a simulation of hard spheres and observe this behavior.

## The integrator

The **Sphere** **integrator** implements HPMC simulations of spheres - Create one:

In [ ]:
mc = hoomd.hpmc.integrate.Sphere()

Set the `shape` *property* to define the **particle shape**.
A sphere is defined by its diameter.

In [ ]:
mc.shape['sphere'] = dict(diameter=1)

## Trial moves

During each **time step**, HPMC attempts `nselect` trial moves on each particle in the system. 
Each **trial move** is drawn from a pseudorandom number stream and may be either a *translation* or *rotation* move.
*Translation moves* displace a particle a random distance (up to `d`) in a random direction.
*Rotation moves* rotate the particle by a random angle about a random axis.
Larger values of `a` lead to larger possible rotation moves.
For hard sphres, HOOMD-blue only attempts translation moves.
It attempts rotation moves for anisotropic shapes.

Any **trial move** whose shape overlaps with another particle is *rejected*, leaving the particle's position and orientation unchanged.
Any **trial move** whose shape *does not* overlap with any other particle is *accepted*, setting the particle's position or orientation to the new value.

`nselect`, `d`, and `a` are *properties* of the integrator:

In [ ]:
mc.nselect = 2
mc.d['sphere'] = 0.15

The `seed` value (passed to the **Simulation** above) controls the sequence of values in the random number stream.
Given the same initial condition and the same `seed`, HPMC simulations will produce exactly the same results.

## Setting the integrator

In [ ]:
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu, seed=1)

An **integrator** is a type of **operation**. There can only be one **integrator** in a **Simulation** and it operates on the system **state** on every **time step**. Assign the HPMC **integrator** to the **Simulation** to use it:

In [ ]:
sim.operations.integrator = mc

Now you have a **Simulation** and a **Sphere integrator**, but can't run the simulation yet.
You first need to define the system **state** for the **integrator** to operate on.
The next section in this tutorial will show you how to initialize the **state**.

# Initializing the System State

## Overview

### Questions

* How do I place particles in the initial condition?
* What units does HOOMD-blue use?

### Objectives

* Describe the basic properties of the **initial condition**, including particle **position**, **orientation**, **type** and the **periodic box**.
* Briefly describe HOOMD-blue's system of **units**.
* Demonstrate writing a system to a **GSD** file.
* Show how to initialize **Simulation state** from a **GSD** file.

The `render_spheres` function in the next (hidden) cell will render the **initial condition** using **fresnel**.

<div class="alert alert-info">
    This is not intended as a full tutorial on <b>fresnel</b> - see the <a href="https://fresnel.readthedocs.io/">fresnel user documentation</a> if you would like to learn more.
</div>

In [ ]:
import fresnel

device = fresnel.Device()
tracer = fresnel.tracer.Path(device=device, w=IMAGE_W, h=IMAGE_W)

def render_spheres(position, L=None):
    scene = fresnel.Scene(device)
    geometry = fresnel.geometry.Sphere(scene,
                                       N=len(position),
                                       radius=0.5)
    geometry.material = fresnel.material.Material(color=fresnel.color.linear([0.01, 0.74, 0.26]),
                                                  roughness=0.5)
    geometry.position[:] = position[:]
    geometry.outline_width = 0.05
    box = fresnel.geometry.Box(scene, [L, L, L, 0, 0, 0], box_radius=.04)
    
    scene.lights = [fresnel.light.Light(direction=(0, 0, 1), color=(0.8, 0.8, 0.8), theta=math.pi),
                    fresnel.light.Light(direction=(1, 1, 1), color=(1.1, 1.1, 1.1), theta=math.pi / 3)]
    scene.camera = fresnel.camera.Orthographic(position=(L * 2, L, L * 2),
                                               look_at=(0, 0, 0),
                                               up=(0, 1, 0),
                                               height=L * 1.4 + 1)
    scene.background_color = (1,1,1)
    return tracer.sample(scene, samples=IMAGE_SAMPLES)

## Components of the system state

You need to initialize the system **state** before you can run a simulation.
The **initial condition** describes the the **position** and **orientation** of every particle in the system and the **periodic box** at the start of the simulation.

The hard sphere system self-assembles the the fcc structure.
Self-assembly is a process where particles will organize themselves into an ordered structure at equilibrium.
Most self-assembly studies run simulations of many thousands of particles for tens of hours.
To keep this tutorial short, it simulations a small number of particles commensurate with the *fcc* structure (`4 * m**3`, where *m* is an integer).

In [ ]:
m = 5
N_particles = 4 * m**3

## Placing particles

In hard particle Monte Carlo, valid particle configurations have no overlaps.
The particle in this tutorial sits is a sphere of diameter 1, so place particles a little bit further than that apart on a *K*x*K*x*K* simple cubic lattice of width *L*.
The following sections of this tutorial will demonstrate how to randomize this ordered configuration and compress it to a higher density.

In [ ]:
spacing = 2
K = math.ceil(N_particles**(1/3))
L = K * spacing

In HOOMD, particle **positions** must be placed inside a **periodic box**.
Cubic boxes range from -*L*/2 to *L*/2.
Use *itertools* and *numpy* to place the positions on the lattice in this range.

In [ ]:
x = numpy.linspace(-L / 2, L / 2, K, endpoint=False)
position = list(itertools.product(x, repeat=3))
print(position[0:4])

Filter this list down to *N* particles because `K*K*K >= N_particles` .

In [ ]:
position = position[0:N_particles]

Here is what the system looks like now:

In [ ]:
render_spheres(position, L)

## Units

HOOMD-blue does not adopt any particular real system of **units**. 
Instead, HOOMD-blue uses an *internally self-consistent* system of **units** and is compatible with *many systems of units*.
For example: if you select the units of *meter*, *Joule*, and *kilogram* for length, energy and mass then the units of force will be *Newtons* and velocity will be *meters/second*.
A popular system of units for nano-scale systems is *nanometers*, *kilojoules/mol*, and *atomic mass units*.

In HPMC, the *primary* **unit** is that of *length*.
*Mass* exists, but is factored out of the partition function and does not enter into the simulation.
The scale of *energy* is irrelevant in athermal HPMC systems where overlapping energies are infinite and valid configurations have zero potential energy.
However, *energy* does appear implicitly in **derived units** like $[\mathrm{pressure}] = \left(\frac{\mathrm{[energy]}}{\mathrm{[length]}^3}\right)$.
In HPMC, $kT$ is assumed to be 1 *energy*.
You can convert a pressure $P$ in HPMC's units to a non-dimensional value with $P^* = \frac{P\sigma^3}{kT}$ where $\sigma$ is a representative length in your system, such as the particle's diameter or edge length.

## Writing the configuration to the file system

[**GSD**](https://gsd.readthedocs.io/) files store the **periodic box**, particle **positions**, **orientations**,  and other properties of the state.
Use the **GSD** Python package to write this file.

In [ ]:
import gsd.hoomd

The **Snapshot** object stores the state of the system.

In [ ]:
snapshot = gsd.hoomd.Snapshot()
snapshot.particles.N = N_particles
snapshot.particles.position = position

Each particle also has a **type**.
In HPMC, each **type** has its own shape parameters.
There is a single **type** in this tutorial, so set a **type id** of 0 for all particles (type ids are 0-indexed):

In [ ]:
snapshot.particles.typeid = [0]*N_particles

Every particle **type** needs a **name**.
Names can be any string.
HOOMD-blue uses the type names to match the parameters you specify in **operations** with the type names present in the **state**.
Give the name `'sphere'` to the type id 0:

In [ ]:
snapshot.particles.types = ['sphere']

**GSD** represents boxes with a 6-element array. Three box lengths *L_x*, *L_y*, *L_z*, and 3 tilt factors. Set equal box lengths 0 tilt factors to define a cubic box.

In [ ]:
snapshot.configuration.box = [L, L, L, 0, 0, 0]

Write this snapshot to `lattice.gsd`:

In [ ]:
with gsd.hoomd.open(name='lattice.gsd', mode='wb') as f:
    f.append(snapshot)

## Initializing a Simulation

You can use the file to initialize the **Simulation state**.
First, create a **Simulation**:

In [ ]:
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu)

Then use the `create_state_from_gsd` factory function to read in the GSD file and populate the **state** with the particles and box from the file:

In [ ]:
sim.create_state_from_gsd(filename='lattice.gsd')

The next section in this tutorial will show you how to use HPMC to randomize the particle positions and orientations.

# Randomizing the System

## Overview

### Questions

* How can I generate a random initial condition?

### Objectives

* Show how to use HPMC to **randomize** the **initial condition**.
* Demonstrate how to **run** a simulation.
* Show how to use HPMC integrator properties to examine the **acceptance ratio**.
* Explain that short simulations at low density effectively randomize the system.

The `render_sphere_snapshot` function in the next (hidden) cell will render a snapshot using **fresnel**.

<div class="alert alert-info">
    This is not intended as a full tutorial on <b>fresnel</b> - see the <a href="https://fresnel.readthedocs.io/">fresnel user documentation</a> if you would like to learn more.
</div>

In [ ]:
import fresnel

device = fresnel.Device()
tracer = fresnel.tracer.Path(device=device, w=IMAGE_W, h=IMAGE_W)

def render_sphere_snapshot(snapshot):
    L = snapshot.configuration.box[0]

    scene = fresnel.Scene(device)
    geometry = fresnel.geometry.Sphere(scene,
                                       N=snapshot.particles.N,
                                       radius=0.5)
    geometry.material = fresnel.material.Material(color=fresnel.color.linear([0.01, 0.74, 0.26]),
                                                  roughness=0.5)
    geometry.position[:] = snapshot.particles.position[:]
    geometry.outline_width = 0.05
    box = fresnel.geometry.Box(scene, snapshot.configuration.box, box_radius=.04)
    
    scene.lights = [fresnel.light.Light(direction=(0, 0, 1), color=(0.8, 0.8, 0.8), theta=math.pi),
                    fresnel.light.Light(direction=(1, 1, 1), color=(1.1, 1.1, 1.1), theta=math.pi / 3)]
    scene.camera = fresnel.camera.Orthographic(position=(0, 0, L+1),
                                               look_at=(0, 0, 0),
                                               up=(0, 1, 0),
                                               height=L+1)
    scene.background_color = (1,1,1)
    return tracer.sample(scene, samples=IMAGE_SAMPLES)

## Method

The previous section of this tutorial placed all the particles on a simple cubic lattice.
This is a convenient way to place non-overlapping particles, but it starts the simulation in a highly ordered state.
You should **randomize** the the system enough so that it forgets this initial state and self-assembly can proceed without influence by the initial condition.

You cannot draw random numbers trivially for the particle positions, as that will result in overlaps between particles.
Instead, start from the lattice and use HPMC to move particles randomly while ensuring that they do not overlap.
In low density configurations, like the lattice generated in the previous section, a short simulation will quickly **randomize** the system.

## Set up the simulation

The following code block creates the **Simulation**, configures the HPMC **integrator**, and initializes the system **state** from `lattice.gsd` as has been discussed in previous sections in this tutorial:

In [ ]:
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu, seed=10)

mc = hoomd.hpmc.integrate.Sphere()
mc.shape['sphere'] = dict(diameter=1)

sim.operations.integrator = mc
sim.create_state_from_gsd(filename='lattice.gsd')

## Run the simulation

Save a snapshot of the current state of the system.
This tutorial uses this later to see how far particles have moved.

In [ ]:
initial_snapshot = sim.state.get_snapshot()

**Run** the simulation to **randomize** the particle positions and orientations.
The `run` method takes the number of steps to run as an argument.
10,000 steps is enough to **randomize** a low density system:

In [ ]:
sim.run(10e3)

You can query properties of the HPMC **integrator** to see what it did.
`translate_moves` is a tuple with the number of accepted and rejected translation moves.
The **acceptance ratio**, the fraction of attempted moves which are accepted, is very high at this low density.

In [ ]:
mc.translate_moves

In [ ]:
mc.translate_moves[0] / sum(mc.translate_moves)

`overlaps` reports the number of overlapping particle pairs in the **state**.
There are no overlaps in the final configuration:

In [ ]:
mc.overlaps

## The final configuration

Look at the final particle positions and see how they have changed:

In [ ]:
final_snapshot = sim.state.get_snapshot()

In [ ]:
render_sphere_snapshot(final_snapshot)

In [ ]:
initial_snapshot.particles.position[0:4]

In [ ]:
final_snapshot.particles.position[0:4]

The particle positions have indeed changed significantly, telling us that the system is well **randomized**.

Save the final configuration to a GSD file for use in the next stage of the simulation:

In [ ]:
hoomd.write.GSD.write(state=sim.state, filename='random.gsd')

The next section of the tutorial takes `random.gsd` and compresses it down to a higher density.

# Compressing the System

## Overview

### Questions

* How do I compress the system to a target density? 
* What is a volume fraction?

### Objectives

* Show how to compute the **volume fraction** of a system.
* Explain how how an **Updater** is an **operation** that modifies the system when its **Trigger** returns `True`.
* Demonstrate using the **QuickCompress** updater to achieve a target volume fraction.
* Demonstrate using the **MoveSize** tuner to adjust the trial move size.

## Volume fraction

Self-assembly in hard particle systems typically occurs at a **volume fraction** above 0.5.
The **volume fraction** is the ratio of the volume occupied by the particles to the volume of the **periodic box**.

So far, this tutorial as **randomized** a system of *N* hard spheres in a box with a very low volume fraction and stored that in `random.gsd`.
Initialize a **Simulation** with this configuration and see what volume fraction it is at:

In [ ]:
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu, seed=10)
sim.create_state_from_gsd(filename='random.gsd')

Compute the volume of one sphere:

In [ ]:
V_particle = 4/3 * math.pi * 0.5**3

Compute the **volume fraction**:

In [ ]:
initial_volume_fraction = sim.state.N_particles * V_particle / sim.state.box.volume
print(initial_volume_fraction)

As you can see, this **volume fraction** is very low and the box volume needs to be significantly reduced to achieve a **volume fraction** above 0.5.

Use HPMC to move particles into non-overlapping configurations while you compress the system.
Set up the HPMC integrator for the hard sphere simulations:

In [ ]:
mc = hoomd.hpmc.integrate.Sphere()
mc.shape['sphere'] = dict(diameter=1)
sim.operations.integrator = mc

## The QuickCompress updater

An **Updater** is a type of **operation** in HOOMD-blue that makes changes to the **state**.
To use an **Updater**, first instantiate the object, assign a **Trigger**, and add it to the **Simulation**.
**Simulation** will apply the **Updater** on **time steps** where the **Trigger** returns `True`.
The **Periodic** trigger returns `True` every `period` steps.

**QuickCompress** is an **Updater** that works with HPMC to quickly compress the box to a target volume.
When triggered, **QuickCompress** reduces the box volume by a scale factor, while *allowing slight overlaps between the particles*.
It then waits for the translation and rotation **trial moves** to remove these overlaps before it reduces the volume again.
This process temporarily produces invalid system configurations, but is much quicker than a process that does not allow temporary overlaps.

Compute the final box size with a **volume fraction** above 0.5 and configure a **QuickCompress** to **trigger** every 10 **time steps**.

In [ ]:
initial_box = sim.state.box
final_box = hoomd.Box.from_box(initial_box)
final_volume_fraction = 0.57
final_box.volume = sim.state.N_particles * V_particle / final_volume_fraction
compress = hoomd.hpmc.update.QuickCompress(trigger=hoomd.trigger.Periodic(10),
                                           target_box = final_box)

Add the **Updater** to the **Simulation**:

In [ ]:
sim.operations.updaters.append(compress)

## The MoveSize tuner

A **Tuner** is another type of **operation**.
**Tuners** make changes to other **operations** to improve performance.
In HPMC, the translation and rotation trial move sizes have a *huge* performance impact.
When the move size is too small it takes many time steps to make appreciable changes to the system.
When the move size is too large very few moves are accepted and it again takes many time steps to make appreciable changes.
The system makes the most progress at moderate move sizes and, in most cases, the optimal **acceptance ratio** is 20%.
The **MoveSize** tuner monitors the **acceptance ratio** and adjusts `d` to achieve the target.

The optimal move size depends on the density of the system.
**QuickCompress** changes the density rapidly during compression, so use the **MoveSize** tuner to adjust the move sizes regularly:

In [ ]:
tune = hoomd.hpmc.tune.MoveSize.scale_solver(moves=['d'],
                                             target=0.2,
                                             trigger=hoomd.trigger.Periodic(10),
                                             max_translation_move=0.2)
sim.operations.tuners.append(tune)

## Run until complete

When the **QuickCompress** updater achieves the target box size and there are no overlaps between particles, the compression process is **complete**.
The number of time steps needed to achieve this varies based on parameters.
Check the `compress.complete` property regularly and stop running the simulation when the compression completes:

In [ ]:
while not compress.complete and sim.timestep < 1e6:
    sim.run(1000)

The `sim.timestep < 1e6` check ensures that this while loop will not waste resources in cases where the compression will never complete.
The loop should complete before that point:

In [ ]:
sim.timestep

Check to see if the compression completed successfully:

In [ ]:
if not compress.complete:
    raise RuntimeError("Compression failed to complete")

The **MoveSize** tuner should have adjusted the move sizes to relatively small values at the final density:

In [ ]:
mc.d['sphere']

Now that the compression is complete, the particles are much closer together and not overlapping, but still arranged randomly.

In [ ]:
render_sphere_snapshot(sim.state.get_snapshot())

Save the final configuration to a GSD file for use in the next stage of the simulation:

In [ ]:
hoomd.write.GSD.write(state=sim.state, filename='compressed.gsd')

# Equilibrating the System

## Overview

### Questions

* What is equilibration?
* How do I save simulation results?

### Objectives

* Explain the process of **equilibration**.
* Demonstrate using **GSD** to write the simulation **trajectory** to a file.
* Demonstrate best practices for move size tuning using **Before** and **And Triggers**.

## Equilibration

So far, this tutorial has placed *N* non-overlapping spheres randomly in a box and then compressed it to a moderate **volume fraction**.
The resulting configuration of particles is valid, but strongly dependent on the path taken to create it.
There are many more **equilibrium** configurations in the set of possible configurations that do not depend on the path.
**Equilibrating** the system is the process of taking an artificially prepared state and running a simulation.
During the simulation run, the system will relax to **equilibrium**.
Initialize the **Simulation** first:

In [ ]:
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu, seed=10)
mc = hoomd.hpmc.integrate.Sphere()
mc.shape['sphere'] = dict(diameter=1)
sim.operations.integrator = mc

The previous section of this tutorial wrote the compressed system to `compressed.gsd`.
Initialize the system **state** from this file:

In [ ]:
sim.create_state_from_gsd(filename='compressed.gsd')

## Writing simulation trajectories

Save the system **state** to a file periodically so that you can observe the equilibration process.
This tutorial previously used **GSD** files to store a single frame of the system **state** using either the **GSD** Python package or `GSD.write`.
The **GSD Writer** (another **operation**) will create a **GSD** file with many frames in a **trajectory**.

In [ ]:
gsd_writer = hoomd.write.GSD(filename='trajectory.gsd',
                             trigger=hoomd.trigger.Periodic(1000),
                             mode='wb')
sim.operations.writers.append(gsd_writer)

## Tuning the trial move size

The previous section used the **MoveSize** tuner regularly during compression to adjust `d` to achieve a target **acceptance ratio** while the system density changed rapidly.
Use it again during the equilibration run to ensure that HPMC is working optimally.

Move sizes should be tuned briefly at the beginning, then left constant for the duration of the run.
Changing the move size throughout the simulation run violates *detailed balance* and can lead to incorrect results.
Trigger the **tuner** every 100 steps but only for the first 5000 steps of the simulation by combining a **Periodic** and **Before** trigger with an **And** operation.
**Before** returns `True` for all **time steps** `t < value` and the **And** trigger returns `True` when all of its child triggers also return `True`.

In [ ]:
tune = hoomd.hpmc.tune.MoveSize.scale_solver(moves=['a', 'd'],
                                             target=0.2,
                                             trigger=hoomd.trigger.And(
                                                 [hoomd.trigger.Periodic(100),
                                                  hoomd.trigger.Before(sim.timestep + 5000)]))
sim.operations.tuners.append(tune)

In [ ]:
sim.run(5000)

Check the acceptance ratios over the next 100 steps to verify that the tuner achieved the target acceptance ratios:

In [ ]:
sim.run(100)

In [ ]:
translate_moves = mc.translate_moves
mc.translate_moves[0] / sum(mc.translate_moves)

## Equilibrating the system

To equilibrate the system, **run** the simulation.
The length of the run needed is strongly dependent on the particular model, the system size, the density, and many other factors.
Hard particle Monte Carlo self-assembly often takes tens of millions of time steps for systems with ~10,000 particles.
This system is much smaller and only takes ~100,000 steps.

Use a **Table Writer** to monitor the progress of the run:

In [ ]:
logger = hoomd.logging.Logger(categories=['scalar', 'string'])
logger.add(sim, quantities=['timestep', 'final_timestep', 'tps'])
table = hoomd.write.Table(trigger=hoomd.trigger.Periodic(period=5000),
                          logger=logger)
sim.operations.writers.append(table)

<div class="alert alert-warning">
This cell will take a few minutes to complete.
</div>

In [ ]:
sim.run(100e3)

Here is the final state of the system after the run.

In [ ]:
render_sphere_snapshot(sim.state.get_snapshot())

Is the final **state** an **equilibrium state**?
The next section in this tutorial shows you how to analyze the **trajectory** and answer this question.

# Analyzing Trajectories

## Overview

### Questions

* How can I analyze trajectories?

### Objectives

* Describe how to access trajectory frames in **GSD**.
* Examine the trajectory with **freud** and **fresnel**.

The `render_spheres_frame` function in the next (hidden) cell will render a snapshot using **fresnel**.
`render_spheres_movie` will render a sequence of frames as an animated GIF.
These methods accept a *particles* argument that filters out which particles to display.

<div class="alert alert-info">
    This is not intended as a full tutorial on <b>fresnel</b> - see the <a href="https://fresnel.readthedocs.io/">fresnel user documentation</a> if you would like to learn more.
</div>

In [ ]:
import fresnel, PIL, IPython, warnings, io, numpy

device = fresnel.Device()
tracer = fresnel.tracer.Path(device=device, w=IMAGE_W, h=IMAGE_W)

def render_spheres_frame(snapshot, particles=None, is_solid=None):
    N = snapshot.particles.N
    L = snapshot.configuration.box[0]
    if particles is not None:
        N = len(particles)
    if is_solid is not None:
        N = int(numpy.sum(is_solid))
    
    scene = fresnel.Scene(device)
    geometry = fresnel.geometry.Sphere(scene,
                                       N=N,
                                       radius=0.5)
    geometry.material = fresnel.material.Material(color=fresnel.color.linear([0.01, 0.74, 0.26]),
                                                  roughness=0.5)
    if particles is None and is_solid is None:
        geometry.position[:] = snapshot.particles.position[:]
    elif particles is not None:
        geometry.position[:] = snapshot.particles.position[particles, :]
    elif is_solid is not None:
        geometry.position[:] = snapshot.particles.position[numpy.ix_(is_solid, [0,1,2])]
        
        
    geometry.outline_width = 0.05
    box = fresnel.geometry.Box(scene, snapshot.configuration.box, box_radius=.04)
    
    scene.lights = [fresnel.light.Light(direction=(0, 0, 1), color=(0.8, 0.8, 0.8), theta=math.pi),
                    fresnel.light.Light(direction=(1, 1, 1), color=(1.1, 1.1, 1.1), theta=math.pi / 3)]
    scene.camera = fresnel.camera.Orthographic(position=(0, 0, L+1),
                                               look_at=(0, 0, 0),
                                               up=(0, 1, 0),
                                               height=L+1)
    scene.background_color = (1,1,1)
    return tracer.sample(scene, samples=IMAGE_SAMPLES)

def render_spheres_movie(frames, particles=None, is_solid=None):
    if is_solid is None:
        is_solid = [None]*len(frames)
    a = render_spheres_frame(frames[0], particles, is_solid[0]);

    im0 = PIL.Image.fromarray(a[:,:, 0:3], mode='RGB').convert("P", palette=PIL.Image.ADAPTIVE);
    ims = [];
    for i,f in enumerate(frames[1:]):
        a = render_spheres_frame(f, particles, is_solid[i]);
        im = PIL.Image.fromarray(a[:,:, 0:3], mode='RGB')
        im_p = im.quantize(palette=im0);
        ims.append(im_p)

    blank = numpy.ones(shape=(im.height, im.width, 3), dtype=numpy.uint8)*255
    im = PIL.Image.fromarray(blank, mode='RGB')
    im_p = im.quantize(palette=im0);
    ims.append(im_p)
            
    f = io.BytesIO()
    im0.save(f, 'gif', save_all=True, append_images=ims, duration=1000, loop=0)

    size = len(f.getbuffer())/1024;
    if (size > 3000):
        warnings.warn(f"Large GIF: {size} KiB")
    return IPython.display.display(IPython.display.Image(data=f.getvalue()))

## Equilibration challenges

In the previous section, you ran the hard sphere system for many time steps to **equilibrate** it and saved the trajectory in `trajectory.gsd`.
Is the final **state** you obtained actually an **equilibrium state**?
Statistical mechanics tells us that as long as our system is *ergodic* it will *eventually* achieve equilibrium.
You need to analyze this trajectory to determine if you have achieved the ordered **equilibrium** structure.

## Tools

There are many tools that can read **GSD** files and analyze or visualize the simulations, and many more Python packages that can work with the numerical data.
This tutorial will show you how to use **freud** to determine which particles are in a solid-like environment and **fresnel** to render system configurations.
The [freud](https://freud.readthedocs.io/) Python package provides a simple, flexible, powerful set of tools for analyzing trajectories obtained from molecular dynamics or Monte Carlo simulations.
The [fresnel](https://fresnel.readthedocs.io/) Python package produces publication quality renders with soft lighting, depth of field and other effects.

While outside the scope of this tutorial, you might want to use tools such as [OVITO](https://www.ovito.org/) or [VMD](https://www.ks.uiuc.edu/Research/vmd/) to visualize the trajectory interactively.
OVITO has built-in support for **GSD** files.
The [gsd-vmd](https://github.com/mphowardlab/gsd-vmd) plugin adds support to VMD.

## Read the trajectory

Use **GSD** to open the **trajectory** generated by the previous section of this tutorial.

In [ ]:
traj = gsd.hoomd.open('trajectory.gsd')

You can index into the frames of the trajectory like a list.
See how many frames exist in the trajectory:

In [ ]:
len(traj)

## Ergodicity

A system is *ergodic* when it can explore all possible states by making small moves from one to another.
In HPMC simulations, low volume fraction simulations are ergodic while very high volume fraction ones are not.
At high volume fractions, there isn't enough free space for the particles to rearrange so they get stuck in local configurations.

Visualize the motion of just a few particles to see if they appear stuck or if they are freely moving about the box:

In [ ]:
render_spheres_movie(traj[::20], particles=[1, 60])

Over the course of the short simulation, individual particles are able to move a few diameters.
This indicates that our system is able to explore phase space and is likely *ergodic*.

## Simulation length

How can you tell if you have run long enough to **equilibrate** the system?
The hard sphere system forms the fcc structure by nucleation and growth.
Nucleation is a rare event, so you need to keep running the simulation until it occurs.
If you ran this simulation many times with different random seeds, each would take a different number of steps to nucleate.
You need to examine the simulation trajectory in detail to determine if you have run it long enough.

In [ ]:
render_spheres_movie(traj[0::12])

Can you see the system ordering?
The particle positions rearrange to line up on evenly spaced planes.

You can use **freud's SolidLiquid** analysis method to quantitatively identify which particles are in the solid structure. Loop over all of the frames in the file and create a boolean array that indicates which particles are in a solid environment:

In [ ]:
solid = freud.order.SolidLiquid(l=6, q_threshold=0.7, solid_threshold=6)
is_solid = []
for frame in traj:
    solid.compute(system=(frame.configuration.box, frame.particles.position),
                  neighbors=dict(mode='nearest', num_neighbors=8))
    is_solid.append(solid.num_connections > solid.solid_threshold)

Plot the total number of particles in a solid environment over time:

In [ ]:
fig = matplotlib.figure.Figure(figsize=(10, 6.18))
ax = fig.add_subplot()
num_solid = [numpy.sum(a) for a in is_solid]
ax.plot(num_solid)
ax.set_xlabel('frame')
ax.set_ylabel('number of particles in a solid environment')
fig

This plot confirms what you saw visually and what you should expect in a system that nucleates and grows a crystal.
There is no solid at the beginning of the simulation.
Then a solid cluster forms and grows quickly to fill the box.

# Spherocylinders in HOOMD

## Overview

### Questions

* How do I run simulations of hard spherocylinders in HOOMD?

### Objectives

* Highlight the code changes needed to convert the hard sphere simulation to model hard spherocylinders.

The `render_spherocylinders` function in the next (hidden) cell will render a snapshot using **fresnel**.

<div class="alert alert-info">
    This is not intended as a full tutorial on <b>fresnel</b> - see the <a href="https://fresnel.readthedocs.io/">fresnel user documentation</a> if you would like to learn more.
</div>

In [ ]:
import fresnel

device = fresnel.Device()
tracer = fresnel.tracer.Path(device=device, w=IMAGE_W, h=IMAGE_W)

def render_spherocylinders(snapshot, params):
    box_L = snapshot.configuration.box[0]

    scene = fresnel.Scene(device)
    geometry = fresnel.geometry.Cylinder(scene,
                                         N=snapshot.particles.N,
                                         radius=params['D']/2)
    geometry.material = fresnel.material.Material(color=fresnel.color.linear([252/255, 209/255, 1/255]),
                                                  roughness=0.5)
    
    top = snapshot.particles.position[:] + rowan.rotate(snapshot.particles.orientation, [0,0,params['L']/2])
    bottom = snapshot.particles.position[:] + rowan.rotate(snapshot.particles.orientation, [0,0,-params['L']/2])
    
    geometry.points[:,0,:] = top
    geometry.points[:,1,:] = bottom
    geometry.outline_width = 0.05
    box = fresnel.geometry.Box(scene, snapshot.configuration.box, box_radius=.04)
    
    scene.lights = [fresnel.light.Light(direction=(0, 0, 1), color=(0.8, 0.8, 0.8), theta=math.pi),
                    fresnel.light.Light(direction=(1, 1, 1), color=(1.1, 1.1, 1.1), theta=math.pi / 3)]
    scene.camera = fresnel.camera.Orthographic(position=(snapshot.configuration.box[0], -snapshot.configuration.box[1]-1, snapshot.configuration.box[2]),
                                               look_at=(0, 0, 0),
                                               up=(0, 0, 1),
                                               height=1.2*(box_L+params['L']))
    scene.background_color = (1,1,1)
    return tracer.sample(scene, samples=IMAGE_SAMPLES)

## Spherocylinders

You can describe a spheroclyinder with a length *L* between end points and the diameter *D*.
Place *N_particles* particles in the at a density of $\rho^*$. $\rho^*$ is the volume fraction relative to the densest possible packing (in the range 0-1) and is commonly used in the spheroclyinder literature.

In [ ]:
L=5
D=1
N_particles = 50
rho_star = 0.53

You can represent a spherocylinder as a 2-vertex spheropolyhedron.

In [ ]:
mc = hoomd.hpmc.integrate.ConvexSpheropolyhedron()
mc.shape['spherocylinder'] = dict(vertices=[[0,0,-L/2],
                                            [0,0,L/2]],
                                  sweep_radius=D/2)

## Setting the initial condition

The spherical caps extend past the end points, so end-to-end spherocylinders must be placed at least a distance of L+D apart.

In [ ]:
snapshot = gsd.hoomd.Snapshot()
snapshot.particles.N = N_particles

spacing = D * 2
K = math.ceil(N_particles**(1/2))
box_L = K * spacing
x = numpy.linspace(-box_L / 2, box_L / 2, K, endpoint=False)
position_2d = list(itertools.product(x, repeat=2))
position_2d = position_2d[0:N_particles]

In [ ]:
snapshot.particles.position = numpy.zeros(shape=(N_particles, 3))
snapshot.particles.position[:,0:2] = position_2d
snapshot.particles.orientation = [1,0,0,0]*N_particles
snapshot.particles.types = ['spherocylinder']
snapshot.configuration.box = [box_L, box_L, 2.0*(box_L + D), 0, 0, 0]

with gsd.hoomd.open(name='lattice.gsd', mode='wb') as f:
    f.append(snapshot)

In [ ]:
render_spherocylinders(snapshot, dict(L=L, D=D))

## Randomizing the system

As with the hard spheres, run the simulation to randomize the particle positions and orientations.

In [ ]:
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu, seed=18)
mc = hoomd.hpmc.integrate.ConvexSpheropolyhedron()
sim.operations.integrator = mc
mc.shape['spherocylinder'] = dict(vertices=[[0,0,-L/2],
                                            [0,0,L/2]],
                                  sweep_radius=D/2)
sim.create_state_from_gsd(filename='lattice.gsd')
sim.run(10e3)
hoomd.write.GSD.write(state=sim.state, filename='random.gsd')

In [ ]:
render_spherocylinders(sim.state.get_snapshot(), dict(L=L, D=D))

## Compressing the system

Use **QuickCompress** to compress the simulation to the target density.
Differences compared to the hard sphere compression code: Computation of the final box, **MoveSize** applies to 'a' moves as well as 'd'.

In [ ]:
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu, seed=18)
mc = hoomd.hpmc.integrate.ConvexSpheropolyhedron()
sim.operations.integrator = mc
mc.shape['spherocylinder'] = dict(vertices=[[0,0,-L/2],
                                            [0,0,L/2]],
                                  sweep_radius=D/2)
sim.create_state_from_gsd(filename='random.gsd')

In [ ]:
rho_c = 2/(math.sqrt(2) + (L/D)*math.sqrt(3))
rho = rho_c * rho_star
box_V = sim.state.N_particles / rho   
box_L=box_V**(1/3)
final_box = hoomd.Box.cube(box_L)

compress = hoomd.hpmc.update.QuickCompress(trigger=hoomd.trigger.Periodic(10),
                                           target_box = final_box)
sim.operations.updaters.append(compress)

tune = hoomd.hpmc.tune.MoveSize.scale_solver(moves=['a', 'd'],
                                             target=0.2,
                                             trigger=hoomd.trigger.Periodic(10),
                                             max_translation_move=0.2,
                                             max_rotation_move=0.2)
sim.operations.tuners.append(tune)   

while not compress.complete and sim.timestep < 5e4:
    sim.run(1000)

hoomd.write.GSD.write(state=sim.state, filename='compressed.gsd')

In [ ]:
render_spherocylinders(sim.state.get_snapshot(), dict(L=L, D=D))

## Equilibrating the system

Tune both 'a' and 'd' move sizes and run the simulation for 200,000 steps.

In [ ]:
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu, seed=18)
mc = hoomd.hpmc.integrate.ConvexSpheropolyhedron()
sim.operations.integrator = mc
mc.shape['spherocylinder'] = dict(vertices=[[0,0,-L/2],
                                            [0,0,L/2]],
                                  sweep_radius=D/2)
sim.create_state_from_gsd(filename='compressed.gsd')

In [ ]:
gsd_writer = hoomd.write.GSD(filename='trajectory.gsd',
                             trigger=hoomd.trigger.Periodic(1000),
                             mode='wb')
sim.operations.writers.append(gsd_writer)

tune = hoomd.hpmc.tune.MoveSize.scale_solver(moves=['a', 'd'],
                                             target=0.2,
                                             trigger=hoomd.trigger.And(
                                                 [hoomd.trigger.Periodic(100),
                                                  hoomd.trigger.Before(sim.timestep + 5000)]))
sim.operations.tuners.append(tune)   

logger = hoomd.logging.Logger(categories=['scalar', 'string'])
logger.add(sim, quantities=['timestep', 'final_timestep', 'tps'])
table = hoomd.write.Table(trigger=hoomd.trigger.Periodic(period=5000),
                          logger=logger)
sim.operations.writers.append(table)

sim.run(200e3)

## Analyze the results

Visualize the final state of the simulation.

In [ ]:
render_spherocylinders(sim.state.get_snapshot(), dict(L=L, D=D))

Compute the nematic order in each frame of the trajectory.

In [ ]:
with gsd.hoomd.open('trajectory.gsd') as traj:
    nematic = freud.order.Nematic([0, 0, 1])
    nematic_order = []
    for frame in traj:
        nematic.compute(frame.particles.orientation)
        nematic_order.append(nematic.order)

fig = matplotlib.figure.Figure(figsize=(10, 6.18))
ax = fig.add_subplot()
ax.plot(nematic_order)
ax.set_xlabel('frame')
ax.set_ylabel('average nematic order parameter')
ax.set_ylim([0, 1]);

In [ ]:
fig